In [4]:
import mnist
import numpy as np
from sklearn.metrics import f1_score

from src.activation import ReLU, SoftMax
from src.classifier import FullyConnected
from src.conv2d import Conv2d
from src.dropout import Dropout
from src.flatten import Flatten
from src.loss import SoftmaxCrossEntropy
from src.nn import NeuralNetwork
from src.optimizer import RAdam
from src.pool2d import Pool2d

In [5]:
def one_hot(x, num_classes=10):
    out = np.zeros((x.shape[0], num_classes))
    out[np.arange(x.shape[0]), x[:, 0]] = 1
    return out


def preprocess(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype(np.float32)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype(np.float32)
    y_train = one_hot(y_train.reshape(y_train.shape[0], 1))
    x_train /= 255
    x_test /= 255
    return x_train, y_train, x_test, y_test

In [6]:
x_train = mnist.train_images()
x_test = mnist.test_images()
y_train = mnist.train_labels()
y_test = mnist.test_labels()

x_train, y_train, x_test, y_test = preprocess(x_train, y_train, x_test, y_test)
print("Dataset successfully downloaded")


cnn = NeuralNetwork(
    input_dim=(28, 28, 1),
    layers=[
        Conv2d(5, 1, 32, activation=ReLU()),
        Pool2d(2, 2, 'max'),
        Dropout(0.75),
        Flatten(),
        FullyConnected(128, ReLU()),
        Dropout(0.9),
        FullyConnected(10, SoftMax()),
    ],
    loss_function=SoftmaxCrossEntropy(),
    optimizer=RAdam
)

cnn.train(x_train, y_train,
          mini_batch_size=256,
          learning_rate=0.001,
          num_epochs=5,
          validation_data=(x_test, y_test))

y_predict = cnn.predict(x_test)
y_predict = y_predict.argmax(axis=1)
print("F1 macro metrics:")
print(f1_score(y_test, y_predict, average='macro'))

Dataset successfully downloaded
Train mode:
  Batch size is 256, learning rate is 0.001

Epoch 1
Done 5.5% / 100%

KeyboardInterrupt: 